In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import requests
from requests_oauthlib import OAuth1
import json
import base64
from PIL import Image
from io import BytesIO

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a childrens therapist for non neural typical children. "
                                      "You describe scenarios to children to help them identify emotion."},
        {"role": "user", "content": "Give me a 2-3 sentence long scenario of a child experiencing joy. "
                                    "Leave out any direct reference to emotion so that we can try to guess it."
                                    "Keep it simple so a 7 year old can guess it."
                                    "I then want to know what are the four top key words that would help identify the emotion."},
    ]
)

print(completion.choices[0].message.content)


Scenario: Tommy just got the toy he had been desperately asking for as a birthday present, and he jumps up and down, clapping his hands at the sight of it.

Key words: toy, birthday present, jumps, clapping


In [4]:
noun_api_key = os.getenv("NOUN_API_KEY")
noun_s = os.getenv("NOUN_S")
auth = OAuth1(noun_api_key, noun_s)
endpoint = "https://api.thenounproject.com/v2/icon?query=dog&limit_to_public_domain=1&limit=1"

response = requests.get(endpoint, auth=auth)
print(response)
response_text = response.content.decode('utf-8')
response_text = json.loads(response_text)
icon_id = response_text["icons"][0]["id"]
print(icon_id)

<Response [200]>
2891610


In [5]:
endpoint2 = "https://api.thenounproject.com/v2/icon/2891610/download?color=A3A3A3&filetype=png&size=25"
response2 = requests.get(endpoint2, auth=auth)

In [6]:
# Check if the request was successful
if response2.status_code == 200:
    # Parse the JSON response
    data = response2.json()
    
    # Extract the base64-encoded file data
    base64_encoded_file = data["base64_encoded_file"]
    
    # Decode the base64-encoded file data
    image_data = base64.b64decode(base64_encoded_file)
    
    # Open the image using Pillow
    image = Image.open(BytesIO(image_data))
    
    # Display the image
    image.show()
    
    # Save the image to a file
    image.save("downloaded_image.png")
    
    print("Image saved successfully.")
else:
    print("Error:", response2.status_code)

Image saved successfully.


In [7]:
print(response2.content)

b'{\n  "base64_encoded_file": "iVBORw0KGgoAAAANSUhEUgAAABkAAAAZCAYAAADE6YVjAAADcUlEQVR4nNWVT2icVRTFz7nvzzfGpBhQolJEJIgLsZsWQdGioEhAESqj8cuELEwWdS2lC0kW3brRhQyNBmYmWQwIcZOdlLZ0pyitWQmtUGNpajRok2nme+9dF53EmmYSA3bRC29133m/d+59f4D7KVSV9Xr9pVqt9uD2nPxfkEajcdR7f1ZEXr4nkIWFhQzAKREhAH9PICsrK6e89y+2222Q/KxWqz1/Z54AUKvV3ibpK5VKc5/rs1arjVlrv0gpUVXhnENRFL+S/CCl9N3o6OjyppMjqnpivw6q1aoFcMI5R1UFAIQQICKPG2MWSH4zPz/fJwAgIoskn5qenu77r4DZ2dljvb29x621x4qiOJtlGYwxUNVbzjmUSiWQ/Kndbq9bADDGfFsURV+pVBqdm5s7n1JSEUlFUSQR2RohBKuqzxljygDesdYihHA6hDCRUjqK200/UxTFWzFG9d5/WS6XIwFgcnJSBgcH5733b25sbESSCuCuoarinMtIIoQAVYX3Hqp6PYTw8cjIyOmdXAsATE1NQTtFFRFD0pJ0JD3JjGSJ5AMikoUQUBQFNnvQbrchIgOq+l630loAaDQaz5AculO8n0gpQVVdt/zm6XrMWmu3A0QEzjkYY/YEkfyrW852FluNMSaSsgkiiRjjtZTSIoCnvfdPqCpijEgp3bUZkot7ObmsqssktxLOOZA8WalUXsuy7HAI4XiM8QyAP51zEPnnsYgxAsCFXSF5nv+hquestf8SkrwCAOVy+Uae55/nef6qiByKMY6klK6LCEQEIYR1kpf2cgKSVzq2YYxBjHGp1Wr9uF0wPDz8c57nswAWSUJVA4Cl/v7+pV0hMzMzJQBvtNvtlqquOOcA4Kvx8fHfdxI1m00DYCCEcE1VLwP4bWhoaGNXiPf+sPf+UErpI